# Ames Housing Step-by-step

Pieter Overdevest  
2024-02-09

For suggestions/questions regarding this notebook, please contact
[Pieter Overdevest](https://www.linkedin.com/in/pieteroverdevest/)
(pieter@innovatewithdata.nl).

### How to work with this Jupyter Notebook yourself?

- Get a copy of the repository [machine-learning-with-python-explainers](https://github.com/EAISI/machine-learning-with-python-explainers) on EAISI's GitHub site. This can be done by either cloning the repo or simply downloading the zip-file. Both options are explained in this Youtube video by [Coderama](https://www.youtube.com/watch?v=EhxPBMQFCaI). Note, the term 'repository' is often abbreviated as 'repo'.

- Copy the Jupyter Notebook 'ames-housing-pieter.ipynb' and the 'utils_pieter' folder to your own project folder.


### Introduction

This case is inspired by Kaggle’s [Getting Started Prediction Competition](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview). The information you need in order to work on this case can be found in the repo
[‘discover-projects/ames-housing’](https://github.com/EAISI/discover-projects/tree/main/ames-housing) on EAISI's GitHub site.
The questions - and therefore this notebook - are structured according to the CRISP-DM framework.

When referring to,

-   'Python Explainer', see the folder 'python-explainers\\'

-   'ML Explainer', see the folder 'ml-explainers\\'

in the repo you just copied.


### Import packages
A package is simply a folder containing modules and a '\_\_init\_\_.py' file, also referred to as a 'dunder init' file ('dunder' referring to *d*ouble *under*scores). A module is just a Python file (*.py) holding functions and variables. The dunder init file is what makes the folder callable as a package. You can include Python code in the dunder init file that will be run when the package is imported. We can import complete packages or only specific functions from a package. In the first case we start with 'import' and in the second case we start with 'from'.

As part of a solution to an exercise, we may need to import specific third party package. These packages will be imported with the solutions. Of some packages we tend to use many functions. In those cases, we simply load the whole package. Typically, we do this under an alias to make it simpler to refer to a function in the concerned package.

In [1]:
# Load packages and assign to a shorter alias.
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# Load packages without an alias, since the package name is already small.
import math
import sklearn 

Over time, I have developed a package containing a number of generally applicable functions. I have named this package 'utils_pieter'. I have included this package in the repo you just copied. If you place the 'utils_pieter/' folder in the same project folder where you copied this Jupyter Notebook 'ames-housing-pieter.ipynb' to, you can import the package by simply stating "import utils_pieter".

In [2]:
# Import modules.
from utils_pieter import f_get_computer_name

Done!


### Notebook settings

In [3]:
# Note, '%matplotlib inline' ensures matplotlib graphs will be included in your notebook.
%matplotlib inline

# Setting Pandas options.
pd.set_option("display.max_rows", 10) # How to display all rows from data frame using pandas. Setting value to None to show all rows.
pd.set_option("display.max_columns", None)
pd.set_option("display.max_info_columns", 100)
pd.set_option("display.max_info_rows", 1000000)
pd.set_option("display.precision", 2)
#pd.set_option("styler.format.precision", 2)

# Setting Matplotlib font sizes.
FONT_SMALL  = 8
FONT_MEDIUM = 16
FONT_LARGE  = 20

plt.rc('axes',   titlesize      = FONT_LARGE)   # axes title
plt.rc('axes',   labelsize      = FONT_MEDIUM)  # axes x and y labels
plt.rc('xtick',  labelsize      = FONT_MEDIUM)  # x tick labels
plt.rc('ytick',  labelsize      = FONT_MEDIUM)  # y tick labels
plt.rc('legend', fontsize       = FONT_MEDIUM)  # legend items
plt.rc('legend', title_fontsize = FONT_LARGE)   # legend title
plt.rc('figure', titlesize      = FONT_LARGE)   # figure title
plt.rc('font',   size           = FONT_SMALL)   # other texts
plt.rc('figure', figsize        = (20, 10))     # figure size. This replaces 'plt.figure(figsize=(20,10))' in every cell.

# Setting sklearn parameter for Pipeline visualization.
# see https://towardsdatascience.com/are-you-using-pipeline-in-scikit-learn-ac4cd85cb27f
sklearn.set_config(display="diagram")

# Confirm sklearn version.
print(f"using scikit-learn version {sklearn.__version__}")

# Seaborn theme.
sns.set_style('whitegrid')

using scikit-learn version 1.2.2


### Project functions

Functions specifically used in this notebook are given below. Functions used across projects are maintained in package 'utils_pieter'.

In [6]:
# Import modules.
from pandas.api.types import is_numeric_dtype

# To sort the neighborhoods by the median of a given numerical column.
# E.g., this function is useful in case you want to order the neighborhoods
# by the median value of 'SalePrices', a numerical variable.
def f_neighborhood_order(df_input, c_col):

    # Error check - Is column 'c_col' numeric?
    if(not is_numeric_dtype(df_input[c_col])):

        print(f"Column {c_col} is not numeric!")
        
        return
 
    v_neighborhood_order = (
        df_input
        .groupby(['Neighborhood'])[c_col]
        .median()
        .sort_values(ascending=True)
        .index
    )

    return list(v_neighborhood_order)

## Let's get started

### Business Understanding

**Business context**: Real estate agency 'Homely Homes' incorporated in Ames, Iowa (USA), needs a good first impression of the sale price of a house as soon as it comes on the market, without having to visit the house. Today, 'Homely Homes' uses their team of real estate agents of different levels of expertise to get an estimate based on the information that is available online. The quality of the estimates differs highly depending on who is asked. And, not surprisingly, the experienced agents are not always readily available. So, the management team of Homely Homes has decided to go full on data, and is requesting you to develop a model that can predict the sale price by the push of a button.

**Business objective**: To become independent on real estate agents to estimate sale prices.

**Scope**: All homes in the city of Ames, IA (USA).

**Project goal**: Develop a model that predicts sale price of a house given a set of it's features,

- The performance metric for your prediction model is the Root Mean Squared Logarithmic Error (RMSLE), i.e., the root mean squared error between the logarithm of the predicted value and the logarithm of the observed sale price. Taking the logarithm means that errors in predicting very expensive houses and cheaper houses will affect the result equally.

- Looking at the [public leaderboard](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/leaderboard), the top 2% have an RMSLE of 0.00044, whilst 25th-percentile and the median performance is at 0.125 and 0.14, respectively.

- As an extra challenge, you can try to trade-off the number of predictors (less is better) vs. performance. Can you make the top 10% (RMSLE 0.123) with the least number of predictors?

## Exercise 1 - Load the 'Ames Housing' dataset

### Data Understanding

There is more than one way to load the 'Ames Housing' dataset. The first two options below load the data directly from a Github repo. The third option can be used in case the data is stored locally.

**1 - Read the data straight from GitHub (A)**

a. Go to the repo [‘discover-projects/ames-housing’](https://github.com/EAISI/discover-projects/tree/main/ames-housing) on EAISI's GitHub site;

b. Click on the 'AmesHousing.csv' file;

c. Click on the **Raw** button;

d. Copy the URL in the URL bar;

e. Paste the URL in the Pandas function 'read_csv()', see below.

In [7]:
df_orig1 = pd.read_csv(
    
    "https://raw.githubusercontent.com/EAISI/discover-projects/main/ames-housing/AmesHousing.csv"
)

**2 - Read the data straight from GitHub (B)**

a. Go to the repo [‘discover-projects/ames-housing’](https://github.com/EAISI/discover-projects/tree/main/ames-housing) on EAISI's GitHub site;

b. Right click (Win) of control click (Mac OSX) on the 'AmesHousing.csv' file;

c. Select 'Copy link';

d. Paste the URL in the Pandas function 'read_csv';

e. Append '?raw=True' to the URL, see below.

In [8]:
# df_orig2 = pd.read_csv(  

#     "https://github.com/EAISI/discover-projects/blob/main/ames-housing/AmesHousing.csv?raw=True",

#     # Just to demonstrate another feature of the `read_csv()` function. In case you want to enforce a
#     # certain data type on a particular column, you can assign a dictionary to the `dtype` argument.
#     # Though, down below we set the types in a different manner.
#     dtype = {
#         'Neighborhood':'category'        
#     }
# )

**3 - Read the data from a local file**

The same Pandas function can be used to read a CSV from your local computer. Note, `f"some text {variable} text continues."` is called an f-string, and allows you to insert variables inside a string.

In [ ]:
#  # Account name of computer.
# c_account = "home" if f_get_computer_name() == "Pieters-MacBook-Pro.local" else "macstudio"

# # Load local data.
# df_orig3 = pd.read_csv(
    
#     f"/Users/{c_account}/Innovate with Data Dropbox/Innovate With Data/Partners/"
#     'PE/2024 02 - EAISI - Discover Projects/ames-housing/AmesHousing.csv'
# )

Since we will make modifications to the data, it is good practice to make a copy, so we can always look back and compare what was in the original data.

In [10]:
df_orig = df_orig1.copy()

## Exercise 2 - Descriptive statistics

### Data Understanding (continued)

We explore the data as received:

#### a. Which variables are numerical? And which are categorical? How many variables do we have of both types?

In [ ]:
df_num_orig    = df_orig.select_dtypes(include='number')
l_df_num_names = list(df_num_orig.columns.values)

print(l_df_num_names)
print(len(l_df_num_names))

The `object` type is a string (categorical) type in Pandas, so we can perform string operations instead of mathematical ones.

In [ ]:
df_cat_orig    = df_orig.select_dtypes(include='object')
l_df_cat_names = list(df_cat_orig.columns.values)

print(l_df_cat_names)
print(len(l_df_cat_names))

Always good to do your book keeping and check your assumptions. Do we consider all columns in either df_num_orig and df_cat_orig? Yes!

In case you are not familiar with f-strings, see "Python 3's f-Strings: An Improved String Formatting Syntax (Guide)" on [realpython.com](https://realpython.com/python-f-strings/).

In [ ]:
print(f"Number of columns in orig data:                            {df_orig.shape[1]}")
print(f"Number of columns in df_num_orig and df_cat_orig combined: {df_num_orig.shape[1]+df_cat_orig.shape[1]}")

#### b. How many missing values do each of the variables have (variable completeness) and what are the variable types? Is `SalePrice` complete? (hint: use `info()`)

The `info()` method shows the variable names and some high level information on each. We observe different data types for integers and floats, i.e., different 'suitcase sizes'. What are dtypes of each variable? What else do we learn from 'Non-Null Count'?

Yes, `SalePrice` is complete. The data frame has 2930 rows and there are 2930 'Non-null' values present in the column.

In [ ]:
# Dimensions of original data frame.
print(df_orig.shape, "\n")

In [ ]:
df_num_orig.info()

In [ ]:
df_cat_orig.info()

#### c. Create a frequency table counting the number of missing values per variable

As we can see above, using `.info()` we can get a quick glance on the variables that have many or no missing values at all. Let's create a simple table listing the variables and their number of missing data, and sort the variables by their 'emptiness' (= 1 - completeness). We conclude that the `Pool QC` variable has missing data in 99.6% of the instances (rows)! The outcome variable - as we could have seen with `.info()` already - has no missing data, as it is not present in the table below.

In [ ]:
# Pandas Series with type of each variable (feature, column) in df_orig.
ps_missing_type    = df_orig.dtypes

# Number of missing data per variable.
ps_missing_total   = df_orig.isnull().sum()

# Percentage of missing per variable.
ps_missing_percent = round(100 * ps_missing_total / df_orig.shape[0], 1)

# Create table (Pandas DataFrame).
df_missing_data = pd.DataFrame(
    {'data_type': ps_missing_type,
     'empty_total': ps_missing_total,
     'empty_perc': ps_missing_percent
    })

# Sort table by number of missing data in descending order.
df_missing_data = df_missing_data.sort_values(by='empty_total', ascending = False)

# Remove variables that have no missing values.
df_missing_data = df_missing_data[df_missing_data.empty_total > 0]

# Show table.
print(f"Number of variables having missing data: {df_missing_data.shape[0]} (out of {df_orig.shape[1]})")
df_missing_data

#### d. Conduct descriptive/summary statistics for numerical variables (e.g., mean, median, std, range) and for categorical variables (e.g., number of unique values, mode, and their frequency)

The `describe()` function outputs some descriptive statistics. For numerical data these stats include: count of non-null values, mean, standard deviation, range (i.e., min and max), the lower quartile, the median, and the upper quartile. Note, `df.describe(include='number')` gives the same output. What can we conclude for the 'SalePrice' variable (min, max, median vs average)?

In [ ]:
df_num_orig.describe()

Fewer descriptive statistics exist for object data. These include: count of non-null values, number of unique values, value with highest frequency, and the frequency at which said value is present. What can we conclude for the 'Street' variable?

In [ ]:
df_cat_orig.describe()

#### e. Extra - An alternative solution to descriptive statistics using `f_describe()`

The `.info()` and `.describe()` functions are built-in Python methods. When you explore data in data frames, you will develop various scripts. In case you apply the same script in two places, you should consider placing that script in a function, allowing you to apply the same script by simply referencing to the name you gave to the function, see also Python Explainer `Functions`.

To pull out a few more descriptive statistics, I developed my own describe function `f_describe()`, which was loaded in memory via `i_general`. It is a kind of a do-all-in-one function to obtain descriptive statistics on all features in the data, numeric and categoric.

Note, it is not mandatory to use keyword arguments. The script `f_describe(df, 10)` results in the same output as `f_describe(df_input = df, n_top = 3)`, since the values are provided in the same order as the keyword argument pairs. For functions with multiple arguments, I recommend you to use keyword arguments to prevent the risk of assigning values to the wrong argument.

In [ ]:
f_describe(df_input = df_orig, n_top = 3)

The `describe()` method only provides the value in each object (categorical) variable present at the highest frequency. Suppose we want to investigate the frequency of all values for an object variable. For that I developed the function `f_freq()`. By default, the top-10 items are shown. In case the feature has more than ten items - eg. `Neighborhood` - you can add input parameter 'n_top' and assign an appropriate number.

In [ ]:
f_freq(df_input=df_orig, c_col='Pool QC')
#f_freq(df_input=df_orig, c_col='Neighborhood', n_top=30)

## Exercise 3 - Impute missing data

Ideally, we move the imputation exercise till after the train/test split. What is the problem of imputing the data before the train/test split?

### Data Preparation

The `info()` method in the previous exercise informed us that there are several missing values in the dataset. These need to be tackled before we can proceed with the remainder of the analysis, in particular before calculating correlations and applying machine learning models. There are many ways to impute missing values, but for now, impute missing values as follows:

a. Impute the number variables with the median value of the available data

b1. Impute the categorical variables with the label "other"

b2. Alternatively, impute the categorical variables with the mode (most frequent value) of the available data

c. Concatenate the numerical and the categorical data into a single data frame

#### a. Impute the number variables with the median value of the available data

We replace NA's in numerical features with the median value in each feature, resp. As expected, we observe that the median values of the updated columns have not changed compared to the orig data. And, there are no empty cells.

In [ ]:
df_num_imputed = df_num_orig.replace(np.nan, df_num_orig.median())

print(df_num_orig.median().head(5))
print("-----")
print(df_num_imputed.median().head(5))
print("-----")
print(df_num_imputed.isna().sum().sum())

#### b1. Impute the categorical variables with the label "other"

Now, we will replace NA's in the columns with categorical data (factor variables). Besides `replace()` [(ref)](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html), we can also make use of `fillna()` [(ref)](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html) to replace NA's by an alternative value. It is good practice to assign an object name to the replacement value ("other"), so it can be changed in one location, if needed.

In [ ]:
c_replace_by = "other"

In [ ]:
# Replace using replace():
df_cat_imputed     = df_cat_orig.replace(np.nan, c_replace_by)

# Replace using fillna():
df_cat_imputed_alt = df_cat_orig.fillna(c_replace_by)

And, as expected there are no empty cells in either of the two data frames.

In [ ]:
print(
    df_cat_imputed.isna().sum().sum(),
    df_cat_imputed_alt.isna().sum().sum()
)

Let's verify the replacement for a given categorical feature (`c_col`).

In [ ]:
# Define feature name to verify the imputations we made.
c_col = 'Pool QC'

# How many NA and 'c_replace_by' exist in the original data:
print("Number of missing values in the original data:")
print("NA: "                + str(sum(df_cat_orig[c_col].isna())))
print(c_replace_by + ": " + str(list(df_cat_orig[c_col]).count(c_replace_by)), "\n")

# How many NA and 'c_replace_by' exist in the updated data - using '.replace':
print("Number of values in updated data using 'replace()':")
print("NA: "                + str(sum(df_cat_imputed[c_col].isna())))
print(c_replace_by + ": " + str(list(df_cat_imputed[c_col]).count(c_replace_by)), "\n")

# # How many NA and 'c_replace_by' exist in the updated data - using '.fillna':
print("Number of values in updated data using 'fillna()':")
print("NA:    "             + str(sum(df_cat_imputed_alt[c_col].isna())))
print(c_replace_by + ": " + str(list(df_cat_imputed_alt[c_col]).count(c_replace_by)))

#### b2. Alternatively, impute the categorical variables with the mode (most frequent value) of the available data

Now, suppose the question was to replace NA's by the most frequent occurring value in each variable. How would we go about this? See also Python Explainer `Mode` in the syllabus.

In [ ]:
df_cat_imputed_most_frequent = df_cat_orig.fillna(df_cat_orig.mode().iloc[0])

In [ ]:
#df_cat_orig.mode().iloc[0].head(5)

In [ ]:
df_cat_orig.describe()

In [ ]:
df_cat_imputed_most_frequent.describe()

#### c. Concatenate the numerical and the categorical data into a single data frame

Next, we need to [concatenate](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html) the two updated data frames to create one new data frame without NA's.

In [ ]:
df_imputed = pd.concat([df_cat_imputed_most_frequent, df_num_imputed], axis=1)

The orig and updated dataframes have the same shape.

In [ ]:
print(df_imputed.shape)
print(df_orig.shape)

#### Extra - Optimize memory usage

The memory taken by the orig data frame is obtained using `info(memory_usage="deep")`.

In [ ]:
df_imputed.info(memory_usage="deep")

Before making any changes, we make a copy of df_imputed, so, we can investigate the effect of memory optimization.

In [ ]:
df_updated = df_imputed.copy()

We convert the `object` columns to `category` columns using [`.astype('category')`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html).

In [ ]:
df_updated[df_updated.select_dtypes(include='object').columns] = df_updated.select_dtypes(include='object').astype('category')

and, the `integer` and `float` columns to their smallest possible versions by using [`pd.to_numeric(downcast=...)`](https://pandas.pydata.org/docs/reference/api/pandas.to_numeric.html#pandas.to_numeric):

In [ ]:
for old, new in [('integer', 'unsigned'), ('float', 'float')]:
    for col in df_updated.select_dtypes(include=old).columns:
        df_updated[col] = pd.to_numeric(df_updated[col], downcast=new)

Let's see what this brings in terms of the size of the data frame.

In [ ]:
df_updated.info(memory_usage="deep")

How much did we reduce the memory need? See also ["How To Get The Memory Usage of Pandas Dataframe?"](https://cmdlinetips.com/2020/03/memory-usage-of-pandas-dataframe/).

In [ ]:
par1 = df_orig.memory_usage(deep=True).sum()/1024/1024
par2 = df_imputed.memory_usage(deep=True).sum()/1024/1024
par3 = df_updated.memory_usage(deep=True).sum()/1024/1024

print(f"Size of original data frame              : {round(par1, 1)} MB.")
print(f"Size of imputed data frame               : {round(par2, 1)} MB.")
print(f"Size of imputed and downcasted data frame: {round(par3, 1)} MB.")
print(f"Reduced original data frame by factor of : {round(par1/par3)}")

In [ ]:
par1 = df_orig1['Neighborhood'].memory_usage(deep=True)/1024/1024
par2 = df_imputed['Neighborhood'].memory_usage(deep=True)/1024/1024
par3 = df_updated['Neighborhood'].memory_usage(deep=True)/1024/1024

print(f"Size of original Pandas series              : {round(par1, 2)} MB.")
print(f"Size of imputed Pandas series               : {round(par2, 2)} MB.")
print(f"Size of imputed and downcasted Pandas series: {round(par3, 3)} MB.")
print(f"Reduced original data frame by factor of    : {round(par1/par3)}")

 In this case the original dataset is not so large (7.8 MB), and we can easily get away with not downcasting the data. If anything gets beyond hundreds of MBs in memory, it helps to optimize. 

## Exercise 4 - Explore the outcome variable (`SalePrice`) and how it correlates to other features

### Data Understanding (continued)

Explore the outcome variable 'SalePrice', representing the sale price of the homes.

#### a. Conduct descriptive/summary statistics on the Y variable (mean, median, std, range)

Since we use the SalePrice data more often, we assign it to a Pandas Series object `ps_y`. What does it mean for the shape of the distribution that the mean exceeds the median?

In [ ]:
ps_y = df_updated['SalePrice']

print("Summary statistics of sale price:")
print(f"Mean:   {round(ps_y.mean(), 1)}")
print(f"Median: {round(ps_y.median(), 1)}")
print(f"SD:     {round(ps_y.std(), 1)}")
print(f"Range:  {ps_y.min()} till {ps_y.max()}")

#### b. Investigate how neighborhood (categorical) and grand living area (numerical) relate to the Y variable. Use, e.g., bar charts, scatter plots, and boxplots (hint: [`seaborn.histplot`](https://seaborn.pydata.org/generated/seaborn.histplot.html), [`seaborn.scatterplot`](https://seaborn.pydata.org/generated/seaborn.scatterplot.html), [`seaborn.boxplot`](https://seaborn.pydata.org/generated/seaborn.boxplot.html), [altair.histogram](https://altair-viz.github.io/gallery/simple_histogram.html), [altair.scatter](https://altair-viz.github.io/gallery/scatter_tooltips.html), [altair.boxplot](https://altair-viz.github.io/gallery/boxplot.html))

In this section, we look at how the `Neighborhood` variable - as example of a categorical feature - and `SalePrice` variable relate to each other. In the same way we look at the relation between the `Gr Liv Area` - as example of a continuous variable - and the `SalePrice`. Like `Neighborhood`, `House Style` is a categorical feature and can be studied in the same way, this is not covered in this notebook.

##### b1. Number of neighborhoods

The function `f_neighborhood_order()` orders the neighborhoods by the median value of a given numerical feature (`c_col`) in the given data (`df_input`). In the example below, we order the neighborhoods by `SalePrice`. The result shows that MeadowV has the lowest median value for `SalePrice` and StoneBr has the highest.

The function `describe()` showed that the `Neighborhood` variable consists of 28 distinct values. This is also observed when determining the length of `l_neighborhood_order_by_saleprice`. So, that's good.

In [ ]:
l_neighborhood_order_by_saleprice = f_neighborhood_order(df_input = df_updated, c_col = 'SalePrice')

print(l_neighborhood_order_by_saleprice)
print(len(l_neighborhood_order_by_saleprice))

Before we look at the differences between neighborhoods, let's see how many houses have been sold in each neighborhood. Seaborn's `countplot()` function plots the frequency of the values in a given categorical feature, i.e., `Neighborhood` in the example below ([ref](https://seaborn.pydata.org/generated/seaborn.countplot.html)). Using `l_neighborhood_order_by_saleprice`, the neighborhoods along the x-axis are ordered by the median of the sale price in the concerned neigherhood. So, we have the expensive areas to the right and the less expensive areas to the left.

In [ ]:
sns.countplot(
    
    data  = df_updated,
    x     = "Neighborhood",
    order = l_neighborhood_order_by_saleprice
);

plt.xticks(rotation=45);
plt.xlabel("Neighborhood")
plt.ylabel("Number of houses");

##### b2. Distribution of Y variable per neighborhoud (Pandas)

What is the benefit of adding `bins` parameter? See below.

In [ ]:
# Local Explainer
np.arange(0, max(ps_y), 20000)

In [ ]:
ps_y.hist(
    
    by      = df_updated['Neighborhood'],
    figsize = (30,20),
    xrot    = 25,
    bins    = np.arange(0, max(ps_y), 20000)
);

##### b3. Distribution of Y variable per neighborhoud (Seaborn)

The `FacetGrid()` function makes life a lot easier..

In [ ]:
# Create grid.
g = sns.FacetGrid(data=df_updated, col="Neighborhood", col_wrap=5)

# Apply 'sns.histplot' to each grid element in g.
g.map(sns.histplot, 'SalePrice');

# Set x-axis ticks.
g.set(xticks=[300000, 600000])

# Manage white space surrounding the subplots.
plt.subplots_adjust(    
    hspace=0.3,
    wspace=0.1
);

##### b4. Scatterplot of Y variable and Grand Living Area (Seaborn)

The plot below shows how seaborn's `scatterplot()` function can directly make use of a categorical feature - `Neighborhood` in this case - to give data points different colors ([ref](https://seaborn.pydata.org/generated/seaborn.scatterplot.html)). Regarding the legend position ([ref](https://www.statology.org/seaborn-legend-position/)).

In [ ]:
c_x="Gr Liv Area"
#c_x="1st Flr SF"

sns.scatterplot(
    
    data = df_updated,  # Before removing homes with high 'Gr Liv Area'.
    #data = df_reduced, # After removing homes with high 'Gr Liv Area'.
    x    = c_x,
    y    = 'SalePrice',
    hue  = 'Neighborhood'
)

plt.legend(bbox_to_anchor=(1.05, 1.1));

Looking at the length of the legend and the broad spectrum of colours, it is clear that this approach is not very informative. Or, we remove the 'hue' parameter. In the next plot we make use of `FacetGrid()` again. Reducing the alpha helps to show the distribution of the data and whether there really is a trend or not. By default - with alpha equal to 1 - ten overlapping data points give the same impression as one data point.

The figure below suggests that houses sold in the more expensive areas - like 'NridgHt' and 'StoneBr' - have a higher sale price per square feet of living area, than in the less expensive areas, like 'MeadowV' and 'NPkVill'.

In [ ]:
# Create a grid.
g = sns.FacetGrid(
    
    data     = df_updated,
    col      = "Neighborhood",
    col_wrap = 5
)

# Apply the sns.histplot to each grid element in g.
g.map(sns.scatterplot, c_x, 'SalePrice', alpha = 0.2)
g.set_xticklabels(rotation=30)

# Manage white space surrounding the subplots.
plt.subplots_adjust(
    
    hspace = 0.5,
    wspace = 1
);

Seaborn's `relplot()` function makes it even easier to plot relations between two features grouped by another feature and colored by yet another feature ([ref](https://seaborn.pydata.org/tutorial/relational.html)).

Houses with more cars in their garages tend to be bigger and have a higher sale prices.

In [ ]:
sns.relplot(
    
    data     = df_updated,
    x        = c_x,
    y        = "SalePrice",
    col      = "Neighborhood",
    hue      = "Garage Cars",
    palette  = "rocket_r",
    col_wrap = 5
);

##### b5. Boxplots of Y variable and Neighborhood (Seaborn)

Boxplots can be made with Seaborn's boxplot() function easily ([ref](https://seaborn.pydata.org/generated/seaborn.boxplot.html)). In the example below, the distribution of the sale price is given for each neighborhood. Note, the neighborhoods are ordered by `l_neighborhood_order_by_saleprice`, the median value - the horizontal line in the middle of the IQR box - goes up steadily from left to right.

Alternatively, we can investigate another continuous variable, like `Gr Liv Area` (grand living area).

In [ ]:
c_col = 'SalePrice'
#c_col = 'Gr Liv Area'

sns.boxplot(
    
    data  = df_updated,
    x     = 'Neighborhood',
    y     = c_col,
    order = l_neighborhood_order_by_saleprice);

plt.xticks(rotation = 25);

##### b6. 2D Histogram of Y variable and Neighborhood

Seaborn's `histplot()` function allows plotting of 2D histograms, when - besides a value for `x` - also a value for `y` is provided ([ref](https://seaborn.pydata.org/generated/seaborn.histplot.html)). The way to *read* this plot is to imagine you view a stack of histograms from above. One histogram for each neighborhood. The higher the frequency the darker the square.

The histplot does not have the option to include an `order` parameter. So, to order the neighborhoods on the y-axis by those in `l_neighborhood_order_by_saleprice`, we use Pandas' `cat.categories` method ([ref](https://pandas.pydata.org/docs/user_guide/categorical.html#operations), [ref](https://stackoverflow.com/questions/38023881/pandas-change-the-order-of-levels-of-factor-type-object)).

In [ ]:
df_updated.Neighborhood = df_updated.Neighborhood.cat.reorder_categories(l_neighborhood_order_by_saleprice, ordered=True)
#l_neighborhood_order_by_saleprice

# Alternatively, create a new data frame:
#df_temp = df_updated.copy()
#df_temp['Neighborhood'] = pd.Categorical(df_temp['Neighborhood'], l_neighborhood_order_by_saleprice)

In [ ]:
sns.histplot(
    
    data   = df_updated, #df_temp

    #x      = "Gr Liv Area",
    x     = "SalePrice",

    y      = 'Neighborhood'
);

##### b7. ECDF's of Y variable and Neighborhood

A different way of looking at distributions is by plotting the cumulative counts using the empirical cumulative distribution function ([ECDF](https://seaborn.pydata.org/tutorial/distributions.html)). This plot draws a monotonically-increasing curve through each datapoint such that the height of the curve reflects the proportion of observations with a smaller value. So, a steeper line means a narrower distribution. The ECDF plot has two key advantages compared to traditional histogram:

1. Unlike the histogram or KDE, it directly represents each datapoint. That means there is no bin size or smoothing parameter to consider, and

2. since ECDF curves are represented by monotonically increasing lined, it is well-suited for comparing multiple distributions.

In the example below, we plot the three least expensive neighborhoods. We observe that 'BrDale' has a narrower `SalePrice` distribution than 'IDOTRR' has.

In [ ]:
# Since Neighborhood is a categorical feature, the legend in each of the
# four graph will show all neighborhoods,  even though only a subset is shown.
# The solution is to bring the feature back to a string. Not to interfere
# with the orig data frame (df), we create a new feature.
df_updated['Neighborhood_str'] = df_updated.Neighborhood.astype(str)

sns.histplot(
    
    data        = df_updated[df_updated['Neighborhood_str'].isin(l_neighborhood_order_by_saleprice[0:3])],
    x           = "SalePrice",
    hue         = "Neighborhood_str",
    element     = "step",
    fill        = False,
    cumulative  = True,
    stat        = "density",
    common_norm = False,
    legend      = True
);

We can do this for all neighborhoods and group them by median sale price. We observe that the more expensive neighborhoods have a wider distribution in sale prices.

In [ ]:
# Prepare the canvas of the subplots.
plt.subplots(
    
    nrows   = 2,
    ncols   = 2,
    figsize = (20,10)
)


# In the first iteration i is equal to 1 and we plot the ECDF plot for the
# first seven neighborhoods in l_neighborhood_order_by_saleprice.
for n_i in range(1, 5):

    # Go to the i-th subplot.
    plt.subplot(2, 2, n_i)

    sns.histplot(
        
        data        = df_updated[df_updated['Neighborhood_str'].isin(l_neighborhood_order_by_saleprice[(n_i-1)*7: n_i*7])],
        x           = "SalePrice",
        hue         = "Neighborhood_str",
        element     = "step", 
        fill        = False,
        cumulative  = True,
        stat        = "density",
        common_norm = False,
        legend      = True,
        binrange    = (0, max(ps_y))
    )

    plt.xticks(rotation=15);


# Manage white space surrounding the subplots.
plt.subplots_adjust(

    left   = None,
    bottom = None,
    right  = None,
    top    = None,
    wspace = None,
    hspace = 0.4
)

Let's remove feature `Neighborhood_str`, since we no longer need it.

In [ ]:
df_updated = df_updated.drop('Neighborhood_str', axis=1)

#### c. Visualize the distribution of the Y variable. What do you observe?

Typically, data distributions are plotted using histograms. The bin width or the number of bins are a means to steer the granularity of the histogram. What is the downside of having too few and of having too many bins?

To make it easier to investigate the effect of the number of bins, we define an object `n_bins`, assign a value to it, and use it with the argument `bins` in the plot functions below.

In [ ]:
n_bins = 50

##### c1. Visualize the distribution of the Y variable - Using Pandas

The Pandas `hist()` function is a quick way to obtain a histogram, by simply appending `.hist()` to a Pandas Series, or to a Pandas Data Frame and specify the column. All three examples below give the same result.

1. How is the third one slightly different?

2. You can see the effect of the ';'?

3. What do observe in the shape of the distribution?

In [ ]:
# Option 1
df_updated['SalePrice'].hist(bins = n_bins);

# Option 2
#ps_y.hist(bins = n_bins);

# Option 3
#df_updated.hist(column = 'SalePrice', bins = n_bins);

##### c2. Visualize the distribution of the Y variable - Using Seaborn

`Seaborn` is a Python data visualization library based on matplotlib ([ref](https://seaborn.pydata.org/index.html)). Below an example using `histplot()` ([ref](https://seaborn.pydata.org/generated/seaborn.histplot.html)).

In [ ]:
sns.histplot(data=df_updated, x="SalePrice", bins=n_bins);

Suppose you want to plot multiple distributions.

In [ ]:
# for x in ['SalePrice', 'Year Built', '2nd Flr SF']:
#     ax = sns.histplot(data=df_updated, x=x, bins=n_bins)
#     plt.show()

##### c3. Visualize the distribution of the Y variable - Estimate number of bins

In the graphs above we assumed a bin size set by `n_bins`. Given the distribution of the observed data we can estimate an optimal bin width using the [Freedman–Diaconis rule](https://en.wikipedia.org/wiki/Freedman%E2%80%93Diaconis_rule):

bin width = 2 * IQR(`x`) / `n`^(1/3)

where, IQR is the InterQuartile Range of the observed data `x`, and `n` is total number of observations in the data. The number of bins can be derived from the bin width and the range. 

For `SalePrice` we arrive at 63 bins.

In [ ]:
# The IQR is equal to the difference between the 75th and 25th percentiles.
n_q25, n_q75 = np.percentile(ps_y, q = [25,75])

n_IQR = n_q75 - n_q25

# The bin width is calculated from the Freedman-Diaconis rule, see above.
n_bin_width = 2 * n_IQR / (len(ps_y)**(1/3))

# The number of bins easily follows from the range and the bin width.
n_bins    = int((max(ps_y) - min(ps_y))/n_bin_width)

print(f"IQR:                              {n_IQR:,.0f}")
print(f"Freedman–Diaconis bin width:      {round(n_bin_width):,.0f}")
print(f"Freedman–Diaconis number of bins: {n_bins}")

### Data Preparation (continued)

##### d. Assess the distribution of `SalePrice` in the previous exercise. What do you observe? Log-transform the outcome variable. What does it mean for the performance of the prediction model?

We observe that the `SalePrice` distribution is skewed to the right. This causes the more expensive homes to 'pull' the predictions to higher values. To solve this we apply log transformation making the distribution more like a normal distribution.

In [ ]:
# We create 'log' brother of ps_y.
ps_y_log = np.log(ps_y)

# Add ps_y_log to df_updated.
df_updated['SalePrice_log'] = ps_y_log

# Add 'SalePrice_log' to l_df_num_names.
l_df_num_names.append('SalePrice_log')

# Let's see the distribution ps_y_log. Looks much better!
sns.histplot(data=ps_y_log, bins=n_bins);

##### e. Assess grand living area ('Gr Liv Area') for all houses in the previous exercise. What do you observe? Remove outliers. What does it mean for the scope of the prediction model?

In the figure above, we observe five houses that have an exceptionally large grand living area (`Gr Liv Area`), while three of them even have a relatively low sale price. We remove al houses with a grand living area exceeding 4,000 sq ft, and limit the scope of the model to houses having a grand living area up to 4,000 sq ft.

We proceed the remainder of the analysis with `df_reduced`. It is good practice to assign a new variable name to the new object, as it has fewer rows, and being a significant change. This keeps `df_updated` available for later reference.  Of course, you cannot create new objects for each change, you will need to find the proper balance, between RAM and clarity.

In [ ]:
# Create a helper list. That is true in case the grand living area is smaller than 4000 sq ft,
# and false in case it exceeds the threshold.
l_temp         = df_updated['Gr Liv Area'] < 4000

# Remove the concerned rows from the Pandas Data frames and from the Pandas Series.
# In addition, we reset the index to prevent issues later on in case of merging data.
df_reduced       = df_updated[l_temp].reset_index(drop=True)
ps_y_log_reduced = ps_y_log[l_temp].reset_index(drop=True)
df_num_reduced   = df_reduced.select_dtypes(include='number').reset_index(drop=True)
df_cat_reduced   = df_reduced.select_dtypes(include='category').reset_index(drop=True)

print("We proceed the analysis with:")
print(f"{df_reduced.shape[0]} rows of the orginal {df_updated.shape[0]} rows in the dataset ({df_updated.shape[1]} columns), and,")
print(f"{len(ps_y_log_reduced)} elements of the orginal {len(ps_y_log)} elements in the outcome variable.")
print(f"While keeping {df_num_reduced.shape[1]} columns in numerical data and {df_cat_reduced.shape[1]} columns in categorical data.")

### Data Understanding (continued)

#### f. Draw scatter plots between Y and each of the numerical features (hint: use [`seaborn.pairplot`](https://seaborn.pydata.org/generated/seaborn.pairplot.html))

Recall that we created a list object `l_df_num_names` to have all numerical variable names readily at hand. We will use this object to create scatterplots of each numerical variable against the SalePrice, using Seaborn's pairplot() function ([ref](https://seaborn.pydata.org/generated/seaborn.pairplot.html)). See also two Stackoverflow references ([ref1](https://stackoverflow.com/questions/31966494/compare-1-independent-vs-many-dependent-variables-using-seaborn-pairplot-in-an-h), [ref2](https://stackoverflow.com/questions/51400076/change-seaborn-pair-plot-figure-size/51403299)).

In [ ]:
print(l_df_num_names)

Let's calculate the number of rows that we need in the grid plot below, given the total number of panels and the number of panels per row, i.e., the number of columns in the grid plot (`n_col`).

In [ ]:
# Initialization.
n_col = 4
n_num = len(l_df_num_names)
n_row = math.ceil(n_num/n_col)

# Number of numerical variables and number of rows, resp.
print(n_num, n_row)

Seaborn's `pairplot()` function allows plotting of a row of scatter plots against the `SalePrice`, by setting `y_vars` equal to `SalePrice`. So, the `for` loop does not have to iterate through each element in `l_df_num_names`. In the example below, we plot `n_col` scatterplots in one row at the time. The setting `kind = 'reg'` adds the trendline, incl. its confidence interval. By default, `kind` is set to `scatter`. To get a better understanding of the data distribution it helps to set a low alpha value (i.e., marker transparancy). We observe that `Overall Quality` correlates well with `SalePrice`. The variable `Total Bsmt SF` also correlates well with `SalePrice`, while we also see some outliers that may cause the trendline to become shallower. Also with other 'SF' variables we observe outliers; something to keep in mind for Data Preparation.

In [ ]:
# In the first iteration i is equal to one. This makes i_start equal to zero and i_stop equal to 4.
# So, l_df_num_names[i_start:i_stop] results in the first four elements in l_df_num_names:
# 'Order', 'PID', 'MS SubClass', and 'Lot Frontage'.

# THIS STEP TAKES A LONGER TIME TO RUN THAN THE OTHERS.
# SO, AS LONG AS I DO NOT NEED IT I LEAVE IT COMMENTED OUT.

# for n_i in range(1, n_row + 1):

#     i_start = (n_i-1) * 4
#     i_stop  = n_i * 4

#     if(i_stop > n_num):
#         i_stop = n_num

#     sns.pairplot(
        
#         data     = df_reduced, # Here, we can offer the complete data frame.
#         y_vars   = ['SalePrice_log'],
#         x_vars   = l_df_num_names[i_start:i_stop], # Here, we determine which variables are plotted.
#         height   = 4,
#         kind     = 'reg',
#         plot_kws = {'line_kws':{'color':'red'}, 'scatter_kws': {'alpha': 0.1}}
#     );

#### g. Draw correlation plots to investigate the correlations between Y and each of the numerical variables (Hint: calculate the Pearson correlation coefficient and use [`seaborn.heatmap`](https://seaborn.pydata.org/generated/seaborn.heatmap.html))

We calculate the Pearson correlation coefficients between numerical features and the outcome variable to understand how they correlate to each other. 

For correlation between categorical and numerical variables you can use ANOVA or the Point Biseral Test ([ref](https://www.tutorialspoint.com/correlation-between-categorical-and-continuous-variables)). For correlation between categorical variables you can use Cramer's V (symmetrical) or Theil's U (asymmetrical) ([ref](https://towardsdatascience.com/the-search-for-categorical-correlation-a1cf7f1888c9)).

##### g1. How is Pearson correlation coefficient calculated?

In [ ]:
# Import module.
from scipy.stats import pearsonr

The Pearson correlation coefficient (named after [Karl Pearson](https://en.wikipedia.org/wiki/Karl_Pearson)) can be used to summarize the strength of the linear relationship between two numerical data samples. To understand the calculation of Pearson's correlation coefficient we reconstruct the calculation ([ref](https://machinelearningmastery.com/how-to-use-correlation-to-understand-the-relationship-between-variables/)). The coefficient is calculated as,

Pearson's correlation coefficient = covariance(X, Y) / (stdv(X) * stdv(Y))

We will work through an example where we calculate the correlation between the features `SalePrice_log` and `Garage Cars`.

In [ ]:
c_feature      = 'Garage Cars'
ps_feature     = df_reduced[c_feature]

m_cov          = np.cov(ps_feature, ps_y_log_reduced)
n_corr_cov_mat = round(m_cov[0,1] / (np.std(ps_y_log_reduced) * np.std(ps_feature)), 3)

n_corr_pearson = round(pearsonr(ps_y_log_reduced, ps_feature)[0], 3)

print(f"Pearson correlation coefficient between feature '{c_feature}' and 'SalePrice_log', calculated using:")

print(f"Covariance matrix-based formula: {n_corr_cov_mat}")

print(f"Python built-in function:        {n_corr_pearson}")


##### g2. Calculate Pearson correlation coefficients between all numerical variables, incl the outcome variable

Now that we have confirmed what `pearsonnr()` calculates, we apply it to all numerical features.

In [ ]:
l_pearson_corr = []

for i in range(0, n_num):

  ps_feature = df_reduced[l_df_num_names[i]]
  
  l_pearson_corr.append(round(pearsonr(ps_y_log_reduced, ps_feature)[0], 2))

  #m_cov = np.cov(ps_feature, ps_y_log_reduced)
  #print(f"{l_df_num_names[i]} : {round(m_cov[0,1] / (np.std(ps_y_log_reduced) * np.std(ps_feature)), 3)} / {round(pearsonr(ps_y_log_reduced, ps_feature)[0], 3)}")

The object `l_pearson_corr` contains the Pearson correlation coefficients between `SalePrice_log` and each numerical feature. We will use it to construct table sorted by the absolute value of the Pearson correlation coefficient (descending).

In [ ]:
# We start out by creating a data frame containing the Pearson correlation coefficients for each of the numerical features.
df_corr_table = pd.DataFrame({'name': l_df_num_names, 'corr': l_pearson_corr})

# We add a feature to df_cov, the absolute value of the correlation coefficient.
df_corr_table['corr_abs'] = abs(df_corr_table['corr'])

# This is allows sorting of the features by their correlation, irrespective of their sign.
df_corr_table = df_corr_table.sort_values(by = 'corr_abs', ascending=False)

# We show the ten numerical variables that have the highest correlation with the SalePrice_log.
df_corr_table.head(20)

We observe that `Overall Qual` has the highest correlation with `SalePrice_log`. What does this means in case of a model predicting `SalePrice_log`?

##### g3. Plot correlation heatmap for all numerical variables

Heatmaps are a very useful way to visualize correlations ([ref](https://heartbeat.fritz.ai/seaborn-heatmaps-13-ways-to-customize-correlation-matrix-visualizations-f1c49c816f07)). To show the benefit of this approach, we plot a heatmap of all correlations among all numerical features. We can identify which pairs have high and which have low correlation. Note, I made function `f_heatmap()` myself, see `i_general.py` to understand how it works. Feel free to copy paste the function into your notebook and check it out. By default, `f_heatmap()` shows the correlation coefficients in each box. Since the boxes are small, we set `b_annotate` to False.

In [ ]:
f_heatmap(    
    df_input           = df_num_reduced,
    v_features_to_show = l_df_num_names,
    b_annotate         = False
)

##### g4. Plot correlation heatmap for Top-10 numerical variables having the heighest correlation with the outcome variable

Using `df_corr_table` we can build a more focused heatmap showing the 10 numerical features that have the highest correlation with `SalePrice_log`. Now, we include the correlation coefficients inside each box for more insights. You can change the font size of the annotation by assigning an argument to the parameter `n_font_size`.

Besides the correlations with `SalePrice_log` it is now also more prominent that `Garage Area` and `Garage Cars` are highly correlated. What does this mean in terms of a linear regression model?

In [ ]:
f_heatmap(    
    df_input           = df_num_reduced,
    v_features_to_show = df_corr_table.head(10)['name'],
    n_font_size        = 20
)

##### g5. Plot correlation heatmap for the numerical variables having 'SF' in the variable name

Let's look at another way of subsetting the numerical features. Suppose we want to select only those features that contain 'SF' ('square feet'). To accomplish that we make use of a list comprehension ([RealPython](https://realpython.com/list-comprehension-python/)), see also the Python Explainer 'List Comprehensions'.

In [ ]:
l_df_num_names_sf = [x for x in l_df_num_names if "SF" in x]

f_heatmap(
    
    df_input           = df_num_reduced,
    v_features_to_show = l_df_num_names_sf
)

Just for demonstration purposes, list comprehensions also allow you to exclude certain strings, say '1st':

In [ ]:
l_df_num_names_sf_wo_1st = [x for x in l_df_num_names if "SF" in x and "1st" not in x or x == 'SalePrice_log']

f_heatmap(
    
    df_input           = df_num_reduced,
    v_features_to_show = l_df_num_names_sf_wo_1st
)

## Exercise 5 - Estimate a Linear Regression, a LASSO and a kNN model

### Modeling

Before diving into LASSO and kNN, we start out by exploring linear regression ([ref](https://towardsdatascience.com/linear-regression-in-python-9a1f5f000606)).

#### a. Estimate a Linear Regression model

We will go through a number of steps, starting with defining a list holding all numerical variables excluding `SalePrice` and `SalePrice_log`. 

##### a1. Define number of objects

Previously, we defined `df_num_reduced`, `df_cat_reduced`, `l_df_num_names`, and, `l_df_cat_names`. Now, we define object `l_df_X_names` as a subset of `l_df_num_names`, excluding `SalePrice`. Later we on, we define object `df_X` as a data frame containing alle numerical features in the data, excluding `SalePrice`.

We confirm that `l_df_num_names` and `l_df_cat_names` make up all features in the original data and that `l_df_X_names` is two items shorter than `l_df_num_names`, excluding `SalePrice` and `SalePrice_log`.

In [ ]:
l_df_X_names = [x for x in l_df_num_names if x not in ['SalePrice', 'SalePrice_log']]

print(
    len(l_df_num_names),
    len(l_df_cat_names),
    df_orig.shape[1],
    len(l_df_X_names)
)

To start out simple we build a linear model considering the Top-4 numerical features with the highest correlation with `SalePrice_log`, the variable that we want to predict. By activating the concerned line we can assign a different list to `l_df_X_names_subset`.

Depending on what scenario you want to follow, add/remove the '#'.

In [ ]:
#l_df_X_names_subset = ['Overall Qual']
#l_df_X_names_subset = ['Overall Qual', 'Gr Liv Area']
#l_df_X_names_subset = ['Overall Qual', 'Gr Liv Area', 'Garage Cars']
l_df_X_names_subset = ['Overall Qual', 'Gr Liv Area', 'Total Bsmt SF']

# Object 'df_X' is data frame containing all numerical features in the data, except 'SalePrice' and 'SalePrice_log'.
df_X = df_reduced[l_df_X_names]

# Define object 'df_X_subset' as subset of df_X.
df_X_subset = df_X[l_df_X_names_subset]

print(df_X.shape, df_X_subset.shape)

**Extra - Select best features using SelectKBest**

For regression tasks, we want to know which features contain the most information i.e., are the best predictors. There are various techniques to answer this question ([ref](https://machinelearningmastery.com/feature-selection-for-regression-data/)). Above, we used the Pearson correlation coefficients and below we will use LASSO. Scikit-learn has various univariate feature selection methods for this purpose ([ref](https://scikit-learn.org/stable/modules/feature_selection.html)). As an example we will briefly demonstrate `SelectKBest()` ([ref](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest)). We observe that the same four features are selected having the highest correlation with `SalePrice`. This should not be a surprise as variance - used for the f-statistic - and correlation are closely related.

In [ ]:
from skl.feature_selection import SelectKBest, f_regression

In [ ]:
# Import modules.
from skl.feature_selection import SelectKBest, f_regression

# For simplification we perform this on the orig dataset. In practice, we would first
# split the data and apply this function to the train data only.
v_3best = (
    
    SelectKBest(f_regression, k = 4)
    .fit(
        df_X, ps_y_log_reduced
    )
    .get_support()
)

# Using get_support() we can filter out the three selected features.
# https://stackoverflow.com/questions/30837803/filter-list-using-boolean-index-arrays
[x for x, b in zip(l_df_X_names, v_3best) if b]

##### a2. Visualize predictors

For completeness we plot the concerned features against `SalePrice`.

In [ ]:
sns.pairplot(
    
    data     = df_num_reduced,
    x_vars   = l_df_X_names_subset,
    y_vars   = 'SalePrice_log',
    kind     = 'reg', # Turns on the trend line
    height   = 7,
    plot_kws = {'line_kws':{'color':'red'}, 'scatter_kws': {'alpha': 0.1}}
);

In [ ]:
# f_heatmap(
#     df_input           = df_num_reduced,
#     v_features_to_show = l_df_X_names_subset
# )

##### a3. Split the data

We split the subset of the predictor data (`df_X_subset`) using `f_train_test_split()` that uses Scikit-Learn's `train_test_split()` function ([ref](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)) and prints some stats, see `i_general.py`.

In [ ]:
# Import module.
from sklearn.model_selection import train_test_split

# Split data in train and test sets.
df_X_train, df_X_test, ps_y_log_train, ps_y_log_test = f_train_test_split(df_X_subset, ps_y_log_reduced)

Let's assess the distribution of the response variable (`ps_y_log_reduced`) in both training and test data. For this, we construct a dataframe of the `ps_y_log_reduced` values and whether they are in the train or test set. Pandas' `set_axis()` function is used to provide the tag feature with a unique index, see also `s_temp2` and `s_temp3` ([ref](https://pandas.pydata.org/docs/reference/api/pandas.Series.set_axis.html)).

In [ ]:
s_temp1 = pd.Series(['train', 'test'])
s_temp2 = s_temp1.repeat([len(ps_y_log_train), len(ps_y_log_test)])
s_temp3 = s_temp2.set_axis(range(len(ps_y_log_reduced)))

df_y_combined = pd.DataFrame({
    
    'SalePrice': pd.concat([ps_y_log_train, ps_y_log_test]),
    'tag':       s_temp3
})

# print(s_temp2.head(5))
# print("")
# print(s_temp3.head(5))
# print("")
# print(df_y_combined.head(5))

The distributions of SalePrice in both train and test data look similar.

In [ ]:
df_y_combined['SalePrice'].hist(
    
    by      = df_y_combined['tag'],
    figsize = (20,10),
    xrot    = 15,
    bins    = 50
);

##### a4. Scaling

The feature data is standardized so that each feature is represented at the same scale. Generally, there are two ways to scale feature data ([ref](https://www.analyticsvidhya.com/blog/2020/04/feature-scaling-machine-learning-normalization-standardization/)):

* Normalization is a scaling technique in which values are translated and rescaled so that they end up ranging between 0 and 1. It is also known as Min-Max scaling.

* Standardization is another scaling technique where the values are centered around the mean with a unit standard deviation ([ref](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)). This means that the mean of the variable becomes zero and the resultant distribution has a unit standard deviation.

There is no rule to tell you what to choose when. You can always start by fitting your model to raw, normalized and standardized data, resp., and evaluate the three outcomes. In the example below, we will standardize the feature data so that each feature will have μ = 0 and σ = 1.

Scaling of outcome variable is generally not required. Only when the outcome variable is not normally distributed, scaling will help to improve model performance.

It is a good practice to fit the scaler on the training data and then use it to transform the testing data. This would avoid any data leakage during the model testing process. Therefore, we use scikit-learn's `fit()` and `transform()` functions in subsequent steps, even though scikit-learn also has a `fit_transform()` function that does both in one go. The difference between `fit()` and `transform()`:
1. 'fit' applies a transformer, like scaling or encoder. The result is an updated 'machine'.
2. 'transform' transforms input data to output data. The updated machine is used to convert raw material (input) to a product (output).

The downside of `fit_transform()` is that in case we apply if before to the train/test split, we leak information from the test data into the training data ([ref](https://towardsdatascience.com/what-and-why-behind-fit-transform-vs-transform-in-scikit-learn-78f915cf96fe)). If we apply it after the train/test split we introduce different means and standard deviations in both train and test data. Applying `Fit()` to the train data allows us to use the resulting mean and standard deviation to standardize the test data.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# Define scaling object.
scaler = StandardScaler()
 
# Standardization of features in training data.
scaler_fitted_on_X_train = scaler.fit(df_X_train)

# Key properties of scaler object.
pd.DataFrame({'name': l_df_X_names_subset, 'mean': scaler_fitted_on_X_train.mean_, 'sd': scaler_fitted_on_X_train.scale_})


With the `scaler` object - that was 'fitted' on the features of the training data - in our hands, we can apply it to transform the orig features in the training data as well as to transform the features in the test data. This approach avoids any information leakage from the test data into the training data.

In [ ]:
df_X_train_scaled = pd.DataFrame(
    scaler_fitted_on_X_train.transform(df_X_train),
    columns = l_df_X_names_subset
)

df_X_test_scaled  = pd.DataFrame(
    scaler_fitted_on_X_train.transform(df_X_test),
    columns = l_df_X_names_subset
)

In [ ]:
# sns.histplot(
    
#     data        = pd.melt(df_X_train_scaled),
#     #data       = pd.melt(df_X_test_scaled),
#     x           = "value",
#     hue         = "variable",
#     legend      = True
# );

##### a5. Train the model

Having completed all preparations, we can train the linear regression model on the train data. It starts by defining `mo_lin_reg` as an empty linear regression model.

In [ ]:
# Import module.
from sklearn.linear_model import LinearRegression

mo_lin_reg = LinearRegression()

What attributes are available in the 'empty' model?

In [ ]:
mo_lin_reg.__dict__

We fill it by fitting the model on the train data. We update the object by fitting the model on the train data (input/output).

In [ ]:
mo_lin_reg.fit(df_X_train_scaled, ps_y_log_train);

Again, we use `__dict__` to see what attributes are available in the `mo_lin_reg` object. Now, we observe the availability of the coefficients (`coef_`) and the intercept (`intercept_`).

In [ ]:
mo_lin_reg.__dict__

##### a6. Interpret the coefficients

We observe that the larger the correlation with the `SalePrice` the larger the fitted coefficient.

In [ ]:
print(f"Intercept: {round(mo_lin_reg.intercept_):,}\n")

print("Coefficients:")

pd.DataFrame({
    'feature': l_df_X_names_subset,    
    'coeff':   [str(round(x,3)) for x in mo_lin_reg.coef_],
    'corr':    df_corr_table.query('name in @l_df_X_names_subset')['corr']
})

##### a7. Make predictions based on estimated model and test data

The fitted model - present in `line_reg` - is used to make `SalePrice` predictions for the test data.

In [ ]:
ps_y_log_pred = mo_lin_reg.predict(df_X_test_scaled)

##### a8. Evaluate estimated model based on test data

There are three primary metrics that can be used to evaluate regression models:

* **Mean Absolute Error (MAE):** The easiest to understand. Represents average error.

* **Mean Squared Error (MSE):** Similar to MAE but noise is exaggerated and larger errors result in higher “punishment”, as the error is squared. MSE is harder to interpret than MAE as it’s not in base units, however, it is generally more popular.

* **Root Mean Squared Error (RMSE):** Most popular metric, similar to MSE, however, RMSE is the square root of the MSE making it more interpretable as it’s in the same unit as the outcome variable. This makes RMSE the recommended metric to interpret your linear regression model.

Since we will evaluate these metrics more often, a function was defined, `f_evaluation_results()`, see 'i_general.py':

In [ ]:
plt.scatter(np.exp(ps_y_log_pred), np.exp(ps_y_log_test));

In [ ]:
f_evaluation_results(ps_y_log_pred, ps_y_log_test)


#### b. Estimate a LASSO model

LASSO (Least Absolute Shrinkage and Selection Operator) is a regression analysis method that performs both variable selection and regularization in order to enhance the prediction accuracy and interpretability of the resulting statistical model ([ref](https://machinelearningmastery.com/lasso-regression-with-python/)). Recall that for regularized linear regression a cost function is added. In case of LASSO this is:

$$ J(\Theta) = MSE (\Theta) + \alpha\sum\limits_{i=1}^n \mid{\Theta_{i}}\mid$$

The larger the hyperparameter $\alpha$ - sometimes referred to as $\lambda$ - the larger the penalty and hence more coefficients will be set to zero.

##### b1. Define number of objects

In addition to the numerical features in the data (`df_X`) you may also want to include categorical data to predict the `SalePrice`. To give an example, earlier, we observed that `Neighborhood` correlated well with `SalePrice`. This can be done by so-called 'One-Hot Encoding' ([ref1](https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/), [ref2](https://towardsdatascience.com/categorical-encoding-using-label-encoding-and-one-hot-encoder-911ef77fb5bd)). In the example below we process the data to add the `Neighborhood` feature to the predictor data. One-hot encoding adds new features to the data, one for each unique value and entering a '1' for observations where it had the concerned value and a '0' in all other cases.

You can see how the `fit_transform()` function is also used here. As in case of scaling, we could perform 'fit' and 'transform' separately. To simplify we do them both in one go.

In [ ]:
# Import module.
from sklearn.preprocessing import OneHotEncoder

# Define a one-hot encoder object.
ohe = OneHotEncoder()

# The fit_transform() function produces a 'sparse matrix' object. To convert the data to
# a dataframe we first need to convert it to a matrix.
sm_neighborhoods = ohe.fit_transform(df_reduced[['Neighborhood']])
m_neighborhoods  = sm_neighborhoods.toarray()

# The one-hot encoder object 'ohe' has been updated by applying the fit_transform() function to it.
# Now it has the 'categories_' attribute. Since it is an array in a list we take the first element.
# Note, we convert the default floats to integers to save memory (more for demo purposes, since our
# dataset is relatively small already).
df_X_ohe = pd.DataFrame(data=m_neighborhoods, columns=ohe.categories_[0]).astype('int')

# Comms to the user. The created data frame has as many features as there are unique values in the
# 'Neighborhood' feature in df_reduced and the same number of observations as df_reduced has.
# No surprises.
print(f"Unique neighborhoods: {ohe.categories_[0]}\n")

print(f"Length of 'categories_' attribute: {len(ohe.categories_[0])}")
print(f"Number of unique neighborhoods:    {len(df_reduced['Neighborhood'].unique())}\n")

print(f"Dimensions of created data frame:  {df_X_ohe.shape}")

Combine the two dataframes horizontally, i.e, 'axis' is set to 1.

In [ ]:
df_X_combined = pd.concat([df_X, df_X_ohe], axis = 1)

# Comms to the user. The created data frame 'df_X_combined' has as many observations as both 'X' and 'X_ohe' have,
# and as many columns as the two have together. 
print("\nThe sum of the number of columns in df_X and df_X_ohe must equal that in df_X_combined:")
print(f"Number of features in df_X:          {df_X.shape[1]}")
print(f"Number of features in df_X_ohe:      {df_X_ohe.shape[1]}")
print(f"Number of features in df_X_combined: {df_X_combined.shape[1]}")

print("\nThe number of rows must all be the same:")
print(f"Number of rows in df_X:              {df_X.shape[0]}")
print(f"Number of rows in df_X_ohe:          {df_X_ohe.shape[0]}")
print(f"Number of rows in df_X_combined:     {df_X_combined.shape[0]}")

##### b2. Visualize predictors

We show a sample of the combined data frame to clarify how one-hot coding works and how the result was added to the numerical data.

In [ ]:
df_X_combined.head(5)

##### b3. Split the data

We split the predictor data using Scikit-Learn's `train_test_split()` ([*ref*](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)), see `i_general.py` for more information on `f_train_test_split()`. We can follow two scenarios:

Scenario A - Use the numerical features only (df_X), so excluding any one-hot encoded categorical features.

Scenario B - Use the combined data (df_X_combined).

Depending on what scenario you want to follow, add/remove the '#'.

In [ ]:
# Scenario A.
#df_X_train, df_X_test, ps_y_log_train, ps_y_log_test = f_train_test_split(df_X, ps_y_log_reduced)

# Scenario B.
df_X_train, df_X_test, ps_y_log_train, ps_y_log_test = f_train_test_split(df_X_combined, ps_y_log_reduced)

##### b4. Scaling

As before, we standardize `df_X_train` and `df_X_test` separately.

In [ ]:
# Define scaling object.
scaler = StandardScaler()
 
# Standardization of features in train data.
scaler_fitted_on_X_train = scaler.fit(df_X_train)

# Key properties of scaler object for first 10 features in X_combined.
pd.DataFrame({'name': df_X_train.columns, 'mean': scaler_fitted_on_X_train.mean_, 'sd': scaler_fitted_on_X_train.scale_}).head(10)

In [ ]:
df_X_train_scaled = pd.DataFrame(
    scaler_fitted_on_X_train.transform(df_X_train),
    columns = df_X_train.columns
)

df_X_test_scaled  = pd.DataFrame(
    scaler_fitted_on_X_train.transform(df_X_test),
    columns = df_X_test.columns
)

Let's have a look at the scaled data. In the one-hot encoded neighborhood features we observe that the 0's and 1's have been replaced by a negative and a positive number, resp.

In [ ]:
df_X_train_scaled.head(5)

In [ ]:
# sns.histplot(
       
#     data        = pd.melt(df_X_train_scaled[['Overall Qual', '1st Flr SF']]),
#     x           = "value",
#     hue         = "variable",
#     legend      = True
# );

##### b5. Train the model

Instead of using Scikit Learn's `Lasso()` function to build a single model, we use `LassoCV()` to build a series of LASSO models. By default `LassoCV()` tries 100 different values for $\alpha$, through the input parameter `n_alphas`. We make use of an example given in SciKit Learn's documentation providing a list of $\alpha$'s ourselves ([ref](https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_model_selection.html#sphx-glr-auto-examples-linear-model-plot-lasso-model-selection-py)).

In [ ]:
# Local Explainer - To explain the list comprehension used below:
#np.arange(-3.5, -0.5, 0.1)
#np.arange(-3.5+0, -0.5, 0.1)
#[round(10**i,3) for i in np.arange(-3.5+0, -0.5, 0.1)]

In [ ]:
# Import module.
from sklearn.linear_model import LassoCV

# Define log of lower border of alphas range.
#n_alphas_min = -3.5

mo_lasso = LassoCV(
    
    # Number of folds.
    cv           = 5,

    # Fixing random_state ensures the results are reproducible.
    random_state = 42,

    # Use any CPU available.
    #n_jobs       = -1,

    # Max number of iterations.
    max_iter     = 1000,

    # We can enforce for which alphas a Lasso model is fitted.
    # In case we do not provide a list, LassoCV() will select 100 values.
    # In case you raise the '0' after to '+' to say '2.5' you select a smaller set of alphas that LassoCV can choose from.
    # This results in larger 'optimal' alpha and we can investigate the effect in section b6. below. 
    #alphas       = [round(10**i) for i in np.arange(n_alphas_min+0, -0.5, 0.1)]

).fit(
    
    df_X_train_scaled,
    ps_y_log_train
)

##### b6. Interpret the coefficients

For each of the folds we plot the RMSE against the respective $\alpha$'s we added as parameter to `LassoCV()`, see colored dotted lines in the figure below. We also add the mean of the folds, see black line. This plot allows us to choose the optimal $\alpha$, i.e., the one that results in the lowest RMSE.

In [ ]:
pd.set_option("display.precision", 5)

df_alpha = pd.DataFrame({
    'alpha':     mo_lasso.alphas_,
    'RMSE_mean': np.sqrt(mo_lasso.mse_path_.mean(axis=-1))
})

df_alpha.sort_values(by = 'alpha').iloc[20:35]

The best model is selected based on the lowest RMSE. The alpha for that model can be obtained through the attribute `alpha_`.

In [ ]:
pd.set_option("display.precision", 2)

print(f"Lowest RMSE found at alpha: {mo_lasso.alpha_:,.5f}")

In [ ]:
# This is to avoid division by zero while doing semilogx on the alphas.
EPSILON = 0.000000001

# Plot RMSE of 5 CV's.
plt.semilogx(
    
    mo_lasso.alphas_ + EPSILON,
    np.sqrt(mo_lasso.mse_path_), ":"
)

# Plot mean RMSE of 5 CV's.
plt.plot(
    
    df_alpha['alpha'] + EPSILON,
    df_alpha['RMSE_mean'],

    color     = "k",
    label     = "Average across the folds",
    linewidth = 2,
)

# Plot vertical dashed line at the alpha value that results in the lowest RMSE.
# Note, the difference between '.alpha_' (below) and '.alphas_' (above).
plt.axvline(
    
    mo_lasso.alpha_ + EPSILON,

    linestyle = "--",
    color     = "k",
    label     = "alpha: CV estimate"
)

plt.legend()
plt.xlabel(r"$\alpha$")
plt.ylabel("Root mean square error")
plt.title("Root mean square error on each fold: coordinate descent")
plt.axis("tight");

From the `mo_lasso` object we can extract the intercept and coefficients of the best model. What explains the number of coefficients equal to zero? How can we increase the number of coefficients equal to zero?

In [ ]:
print(f"intercept: {mo_lasso.intercept_:,.2f}")

df_lasso_coefficients = pd.DataFrame(

    {
        'name':            df_X_train_scaled.columns,
        'lasso coeff':     mo_lasso.coef_,
        'lasso_coeff_abs': abs(mo_lasso.coef_)        
    }

).sort_values(

    'lasso_coeff_abs',
    ascending=False
)

df_lasso_coefficients

##### b7. Make predictions based on estimated model and test data

The `mo_lasso` object holds the properties of the best model, i.e., the one resulting in the lowest RMSE.

In [ ]:
ps_y_log_pred = mo_lasso.predict(df_X_test_scaled)

##### b8. Evaluate estimated model based on test data

The same three primary metrics are used to evaluate the best LASSO model. The RMSE is considerably lower, however, we require a lot more features. Using $\alpha$ we can reduce the number of included features, but this goes at the 'expense' of a higher RMSE.

In [ ]:
f_evaluation_results(ps_y_log_test, ps_y_log_pred)

#### c. Estimate a kNN model

In addition, we develop a kNN model to predict the `SalePrice` ([ref](https://realpython.com/knn-python/)). We define a KNeighborsRegressor object, called `knn_model`, that is further informed by fitting the training data. Contrary to (LASSO) regression, we keep this section on kNN simple and to the point; you got the idea.

In [ ]:
# Import module.
from sklearn.neighbors import KNeighborsRegressor

# Define KNN object.
mo_knn = KNeighborsRegressor(n_neighbors = 2)

mo_knn.fit(df_X_train_scaled, ps_y_log_train);

The trained kNN model is used to predict `SalePrice` for the train and test data. We do this to investigate overfitting.

In [ ]:
f_evaluation_results(ps_y_log_train, mo_knn.predict(df_X_train_scaled))

In [ ]:
f_evaluation_results(ps_y_log_test, mo_knn.predict(df_X_test_scaled))

For low values of `n_neighbors` the model is highly flexible and we are running the risk of over-fitting. Observing the RMSE value for the train and test set, we can conclude that we are indeed overfitting. We can make use of SciKit Learn's `GridSearchCV()` function to work through a series of hyperparameters, and determine for which hyperparameter we found the best model.

In [ ]:
# Import module.
from sklearn.model_selection import GridSearchCV

parameters = {"n_neighbors": range(1, 50)}
gridsearch = GridSearchCV(KNeighborsRegressor(), parameters)
gridsearch.fit(df_X_train_scaled, ps_y_log_train)

# Comms to the user
print(f"We found the best model for {gridsearch.best_params_}.")

In [ ]:
f_evaluation_results(ps_y_log_train, gridsearch.predict(df_X_train_scaled))

In [ ]:
f_evaluation_results(ps_y_log_test, gridsearch.predict(df_X_test_scaled))

Indeed, no over-fitting.

## Exercise 6 - Assess which model performs best

### Evaluation

This is discussed with each of the models. I leave making the overall assessment to you ;-)

## Appendices

### Appendix A - Use Pipelines to model Ames Housing data with LASSO

In this section, we repeat - more or less - what we did above, but now using pipelines to demonstrate their use. See also Python Explainer 'pipelines' in the syllabus. Besides the functions `Pipeline()` or `make_pipeline()` we make use of the `ColumnTransformer()` function to process numerical and categorical features differently. We start out by defining separate pipelines for numerical and categorical features. Here, we use the `Pipeline()` function, instead of the `make_pipeline()` function to make it easier to reference to individual transformers later on. Imputation is performed by SciKit Learn's `SimpleImputer()` function ([ref](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html)). 

In [ ]:
# Import module.
from sklearn.impute import SimpleImputer

from sklearn.pipeline import Pipeline, make_pipeline

In [ ]:
# Create pipeline for numerical features.
pl_Num = Pipeline([

        ( "impute", SimpleImputer(missing_values = np.nan, strategy = "median") ),

        ( "scale",  StandardScaler() )
])

# Create pipeline for categorical features.
pl_Cat = Pipeline([

        ( "impute", SimpleImputer(missing_values = np.nan, strategy = "most_frequent") ),

        ( "onehot", OneHotEncoder() )
])

The parameter `transformers` in the function `ColumnTransformer()` receives a list of tuples each containing: (1) transformer name, (2) transformer, and (3) feature name(s). Each tuple specifies how the specified features in the concerned tuple are transformed ([ref](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html)).

Note, `remainder = 'drop'` tells the transformer to drop the features not mentioned in the tuples.

In [ ]:
# Import module.
from sklearn.compose import ColumnTransformer

# Define transformer object.
pl_ColumnTransformer = ColumnTransformer(
    
    transformers = [
        
        # Tuples containing transformer name, transformer, and feature names for each transformation to take place.
        ('num', pl_Num, l_df_X_names),

        # As we pull the data through a pipeline, it is easier to add more categorical features.
        ('cat', pl_Cat, ['Neighborhood'] + ['Pool QC'])
    ],
    
    remainder = 'drop',
    verbose   = True
)

As with scaling and one-hot encoding we use the `fit_transform()` function to pull a data frame through the pipeline. While with scaling we perform `fit()` and `transform()` separately, to apply the scaling based on the train set to the test set, with one-hot encoding we perform both steps in one go by using `fit_transform()`. The latter approach we apply to the original data, `df_orig`, resulting in `m_X_transformed`.

Do the number of columns in `m_X_transformed` correspond to what we expect?

Why are we not able to do the calculation for 'Pool QC' in the same notation as for 'Neighborhoord'? 

In [ ]:
m_X_transformed = pl_ColumnTransformer.fit_transform(df_orig)

print('')
print(f"Number of columns in the resulting array:         {m_X_transformed.shape[1]}")
print(f"Number of numerical features (excl. 'SalePrice'): {len(l_df_X_names)}")
print(f"Number of unique values in 'Neighborhood':        {len(df_orig['Neighborhood'].value_counts())}")
print(f"Number of unique values in 'Pool QC':             {len(df_orig['Pool QC'].value_counts())}")

Next on our list of activities is to create a list of feature names, so we can convert the array `m_X_transformed` to a data frame and assign the corresponding feature names. For the numerical part this is easy, as the numerical feature names are stored in `l_df_X_names`. 

For the one-hot encoded features - `Neighborhood` and `Pool QC` - this is a bit trickier. The object `pl_ColumnTransformer` contains the attribute `named_transformers_`. This has two elements, 'num' and 'cat', i.e., the names we gave to the respective transformers. We see the benefit of `Pipeline()` over `make_pipeline()`, since we can make use of the names that we gave to each pipeline. With `make_pipeline()` we do not (have to) give names to the individual transformers, Python creates them for us. However, this means that it is more cumbersome to refer to individual transformers, if needed at later stage, as we see here. See also Python Explainer 'pipeline' in the syllabus.

We assign the feature names derived from `pl_ColumnTransformer` to object `v_df_cat_transformed_names`, and we observe that they match what we derived above.

In [ ]:
v_df_cat_transformed_names = pl_ColumnTransformer.named_transformers_['cat']['onehot'].get_feature_names_out(['neighborhood', 'pool_qc'])

print(v_df_cat_transformed_names)

We continue and append the numeric feature names and those that follow from the two categorical features, `Neighbordhood` and `Pool QC`.

In [ ]:
v_df_X_transformed_names = np.append(l_df_X_names, v_df_cat_transformed_names)

Let's do our checks and balances to confirm the dimensions of our data is mathing our expectations, see below. The transformed data matrix `m_X_transformed` consists of as many columns as there are elements in `v_df_X_transformed_names` that we created from the numerical features in `df_X` and the unique values in `Neighborhood` and `Pool QC`.

In [ ]:
print(f"Number of columns in the resulting array: {m_X_transformed.shape[1]}")
print(f"Length of 'v_df_X_transformed_names':     {len(v_df_X_transformed_names)}")


Now, we are ready to construct data frame `df_X_transformed` that follows from the ColumnTransformer.

In [ ]:
# Convert the 2D array to a data frame:
df_X_transformed = pd.DataFrame(m_X_transformed, columns = v_df_X_transformed_names)

df_X_transformed.head(5)

We split the predictor data using Scikit-Learn's `train_test_split()` ([*ref*](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)).

In [ ]:
df_X_train, df_X_test, ps_y_log_train, ps_y_log_test = f_train_test_split(df_X_transformed, ps_y_log)

Instead of using Scikit Learn's `Lasso()` function to build a single model, we use `LassoCV()` to build a series of LASSO models. By default `LassoCV()` tries 100 different values for $\alpha$, through the input parameter `n_alphas`. We make use of an example given in SciKit Learn's documentation providing a list of $\alpha$'s ourselves ([ref](https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_model_selection.html#sphx-glr-auto-examples-linear-model-plot-lasso-model-selection-py)).

In [ ]:
# Define log of lower border of alphas range.
#n_alphas_min = 2

mo_lasso = LassoCV(
    
    # Number of folds.
    cv           = 5,

    # Fixing random_state ensures the results are reproducible.
    random_state = 42,

    # Use any CPU available.
    #n_jobs       = -1,

    # Max number of iterations.
    max_iter     = 100000,

    # We can enforce for which alphas a Lasso model is fitted.
    # In case we do not provide a list, LassoCV() will select 100 values.
    #alphas       = [round(10**i) for i in np.arange(n_alphas_min, n_alphas_min+3, 0.2)]

).fit(
    
    df_X_train,
    ps_y_log_train
)

From the `mo_lasso` object we can extract the intercept and coefficients of the best model. What explains the number of coefficients equal to zero? How can we increase the number of coefficients equal to zero?

In [ ]:
print(f"intercept: {mo_lasso.intercept_:,.0f}")

pd.DataFrame(

    {
        'coef':     mo_lasso.coef_,
        'coef_abs': abs(mo_lasso.coef_),
        'feature':  df_X_train.columns
    }

).sort_values(

    'coef_abs',
    ascending=False
)

The best model is selected based on the lowest RMSE. The alpha for that model can be obtained through the attribute `alpha_`.

In [ ]:
print(f"Lowest RMSE found at alpha: {mo_lasso.alpha_:.2f}")

The `mo_lasso` object holds the properties of the best model, i.e., the one resulting in the lowest RMSE.

In [ ]:
ps_y_log_pred = mo_lasso.predict(df_X_test)

The same three primary metrics are used to evaluate the best LASSO model. The RMSE is considerably lower than when we limited the predictor data to numerical features only, see above. We observe that by adding `Neighborhood` to the model it can explain a larger part of the variance in the data.

In [ ]:
f_evaluation_results(ps_y_log_test, ps_y_log_pred)

###  Appendix B - Determine number of principle components to explain 90% of variance in the data

In this section, we determine the number of principle components (PC) to explain 90% of the variance in the data using a pipeline ([ref](https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60)), see also 'ML Explainer' 'pca'. So, we need access to the `explained_variance_ratio_` attribute of a PCA object. As far as I know, this can only be done outside the pipeline. In case we are only interested in the principle components themselves, we can include `PCA(n_components = ...)` in the pipeline.

In [ ]:
pl_impute_scale_numerical = make_pipeline(
    
    SimpleImputer(missing_values = np.nan, strategy = "median"),

    StandardScaler()

    )

The pipeline is applied to the numerical features in the orig data (`df_orig`).

In [ ]:
m_X_transformed = pl_impute_scale_numerical.fit_transform(df_orig[l_df_X_names])

We define a PCA object `pca_` anticipating that 25 principle components will cover at least 90% of the variance.

In [ ]:
# Import module.
from sklearn.decomposition import PCA

# Define PCA object.
pca_ = PCA(n_components=25)

# Matrix containing the principle components.
m_pc = pca_.fit_transform(m_X_transformed)

The attribute `explained_variance_ratio_` holds the additional variance that is explained by adding another principle component.

In [ ]:
v_pca_ames = pca_.explained_variance_ratio_
v_pca_ames

Using a list comprehension, we demonstrate that 23 PC's are needed to explain at least 90% of the total variance in the data. In other words the 23 PC's contain 90% of the information present in the 38 features. To show the outcome of `np.cumsum(v_pca_ames)`:

In [ ]:
print(len(l_df_X_names))

np.cumsum(v_pca_ames)

In [ ]:
[[i,j] for i,j in enumerate(np.cumsum(v_pca_ames))]

Let's put the following in one table to compare the order of the numerical features in the data, see table below:

1. **Pearson Correlation coefficients** of the numerical features with `SalePrice`.

2. **LASSO coefficients** fitting the numerical features to a LASSO model predicting `SalePrice`.

3. **PCA loadings** of first the principle component ('PC1'). Note, this is independent of `SalePrice`.

Two housekeeping remarks:
1. We make use of `reset_index(drop=True)` to concatenate the data frames as there are offered. Without it, the data frames would be joined using the orig index and all feature names would end up in the same row driven by the index of the first data frame, the correlation coefficients in this case.

2. In 'Step 3 - Split the data' in section 'Estimate a LASSO model' two scenario's can be chosen (A and B). Subsequently, in step 6 of the same section, `df_lasso_coefficients` is calculated. To allow for a proper comparison of the correlation coefficients, the LASSO coefficients, and the PCA loadings, please ensure you ran scenario A and steps 3-6 accordingly.

The table below allows us to investigate the numerical feature order in each of the three analysis. Comparing the Pearson correlation coefficients and the LASSO coefficients shows that the first two features occur at the top of each list. We also expect features that have a high correlation with the `SalePrice` to also end up high in the LASSO coefficient table. *Question: Why?* However, the equality does not continue all the way down. `Garage Area` is high in the list of correlation coefficients, however, it is somewhere in the middle in the list of LASSO coefficients. *Why is this the case?* Since LASSO wants to include as few features as possible (regularization, remember $\alpha$) it will choose one of the two. Both are highly correlated (not shown in the table), so the information is sufficiently captured in one of the two.  We observe the same for `Gr Liv Area` and `1st Flr SF`. The table suggests that the feature with the higher correlation with `SalePrice` is chosen for LASSO and the other one is *punished* by giving it a lower LASSO coefficient. So, correlations between features causes the order in the two lists to differ.

We observe no similarity between the order in features between the loadings in the first principle component (PC1) on the one hand and the correlations and LASSO coefficients on the other hand. Possibly, this is explained by PCA only focussing on the predictor data (X), where correlations and LASSO depend on the relation between the predictor data (X) and `SalePrice` (y).

In [ ]:
if df_lasso_coefficients.shape[0] != 38:
    print("Before proceeding, run Scenario A ('numerical only') in Step 3 of section 'Estimate a LASSO model', and run steps 3-6 above.")

In [ ]:
pd.concat(
        
    [   # Correlation coefficients. Note, we remove the first row containing correlation of SalePrice with itself.
        df_corr_table.tail(-1).reset_index(drop=True),

        #LASSO coefficients.
        df_lasso_coefficients.reset_index(drop=True),
          
        # PCA Loadings.  
        pd.DataFrame({
            
            'name': l_df_X_names,
            'pc1_loading': ["{:.2f}".format(x) for x in pca_.components_[1]],
            'pc1_loading_abs': ["{:.2f}".format(abs(x)) for x in pca_.components_[1]]

        }).sort_values(
            
            by = 'pc1_loading_abs',
            ascending = False
            
        ).reset_index(drop=True)
    ],
    
    axis = 1
)

## Exercise 7 - Interpretable Machine Learning - Using SHAP values to explain contribution of features to prediction of Sale Price

Some suggestions to enter in chatGPT:
* How to import random forrest regression model in python?
* What hyperparameters do I have access to in this model?
* Can you suggest a dictionary with some common ranges to use with these hyperparameters?
* How to use this model in conjunction with gridsearchcv?

Of course, these questions are already quite specific and when you start your questions might be more generic, like "How do I do check the performance on a range of hyperparameters". The examples above show that also as you develop your knowledge, ChatGPT remains a very good source for suggesting snippets of code.

### Exercise A - Construct a data frame holding the imputed numerical data of the Ames Housing data set. Optionally, as a challenge, also include the one hot encoded neighborhoods. Perform a train/test split on the data frame you constructed. Since, the SHAP calculation are computer intensive, take the first 500 observations from the resulting training set (call it, `df_X_fraction`) and take the first 500 elements of the outcome variable from the resulting training set (call it, `ps_y_fraction`).

In [ ]:
# One way to obtain the two objects is to run scenario B in Exercise 5.3b

# We take a fraction from the data to speed up the exercise. SHAP analysis are computer intensive.
df_X_fraction = df_X_train.iloc[0:500]
ps_y_fraction = ps_y_log_train[0:500]

### Exercise B - Copy/paste the content from the cell below to your notebook.

In [ ]:
# Import module.
from sklearn.ensemble import RandomForestRegressor


# Hyperparameter grid:
dc_hyperparameter_ranges = {

    'bootstrap': [True,False], # Do we bootstrap samples, or not.
    'max_depth': [20],         # Maximum depth of each tree
    'min_samples_leaf': [4],   # Minimum number of samples required to be at a leaf node
    'min_samples_split': [4],  # Minimum number of samples required to split an internal node
    'n_estimators': [1000],    # Number of trees
    'max_features': ['auto'],  # Number of features to consider at each split
    'random_state': [42]       # Random state for reproducibility
}

# Perform a gridsearch on the random forest model:
gridsearch = GridSearchCV(
    estimator  = RandomForestRegressor(),
    param_grid = dc_hyperparameter_ranges,
    scoring    = 'neg_mean_squared_error',
    cv         = 5
)

# Use subset of training data to do a gridsearch on the random forest model:
gridsearch.fit(df_X_fraction, ps_y_fraction)

### Exercise C - Show the value of the `best_params` attribute of the `gridsearch` object. What do the attributes `best_params_` and `best_estimator_` refer to? Assign the value of the `best_estimator_` attribute to a new object called, `best_model`.

In [ ]:
# Best parameters from the set of hyperparameters.
gridsearch.best_params_

In [ ]:
# This is the best model. So, we do not need to rerun the model with the best parameters.
best_model = gridsearch.best_estimator_

### The two cell below are extra (no exercise).

In [ ]:
f_evaluation_results(ps_y_fraction, gridsearch.predict(df_X_fraction))

In [ ]:
#Model evaluation
plt.figure(figsize=(5, 5))

plt.scatter(ps_y_fraction, gridsearch.predict(df_X_fraction))
plt.plot([9, 14], [9, 14], color='r', linestyle='-', linewidth=2)
plt.xlabel('Actual Test',size=20)
plt.ylabel('Predicted Test',size=20);

### Exercise D - Copy/paste the content from the cell below to your notebook.

In [ ]:
import shap
shap.initjs()

# Create SHAP object.
explainer = shap.Explainer(best_model)

# Create SHAP values.
shap_values = explainer.shap_values(df_X_fraction)

### Exercise E - Waterfall Plot

We can use the waterfall function to visualise the SHAP values of one observation. Copy/paste the content from the cell below to your notebook. The questions refer to this code cell and to the resulting plot.

1 - Define an object to which you assign the index of the data point you want to explain the prediction for. Assign the value to your object such that you can explain the prediction for the first observation in the data.

2 - Replace '...' by the object name you defined above.

3 - What is the value for explainer.expected_value[0]?

4 - Run the cell.

5 - What do you conclude from the resulting figure? Use: (1) the answer from question 3 and (2) the prediction for the first observation in the data.

For reference see also [API Reference of SHAP module](https://shap.readthedocs.io/en/latest/generated/shap.plots.waterfall.html).

In [ ]:
# Set index of data point you want to explain the prediction for.
index = 1

# Plot waterfall
shap.plots.waterfall(
    
    shap.Explanation(
        base_values   = explainer.expected_value[0], # Mean prediction for the entire training data.
        values        = shap_values[index],          # Subset of shap values.
        data          = df_X_fraction.iloc[index],   # Subset of training data.
        feature_names = df_X_fraction.columns        # Feature names.
))

In [ ]:
# ANSWERS

# 1 - index = 1

# 2 - index

# 3 - It is the mean prediction for the entire training data. The answer should be a number; in my case it is 12.01.
# Though in individual cases the number may differ because we sampled the data differently.

# 5 - There will be a unique waterfall plot for every observation in our dataset. They can all be interpreted in
# the same way as above. In each case, the SHAP values tell us how the features have contributed to the prediction
# when compared to the mean prediction. Large positive/negative values indicate that the feature had a significant
# impact on the model’s prediction. The prediction of 12.27 (in my case) can be constructed from the base value of
# 12.01 (mean prediction) and the contributions of the individual features in the data, e.g., the value of 0.6416
# for Overal Qual (in my case) add almost 0.26 to the prediction for the log of the sales price.

### Exercise F - Force Plot

Another way to visualise these is using a force plot. You can think of this as a condensed waterfall plot. Copy/paste the content from the cell below to your notebook. The questions refer to this code cell and to the resulting plot.

1 - The force plot is a different representation of the waterfall plot. Apply the questions from exercise 1 to the cell below.

In [ ]:
# Set index of data point you want to explain the prediction for. Write the answer to question a, below.
index = 1

# Plot Force Plot.
shap.plots.force(

    base_value    = explainer.expected_value[0], # Mean prediction for the entire training data.
    shap_values   = shap_values[index],          # SHAP values.
    features      = df_X_fraction.iloc[index],   # Training data.
    feature_names = df_X_fraction.columns        # Feature names.
)

In [ ]:
# ANSWERS

# See Exercise E.

### Exercise G - Stacked Force Plot

Waterfall and force plots are great for interpreting individual predictions. To understand how our model makes predictions in general we need to aggregate the SHAP values. One way to do this is by using a stacked-force plot. We can combine multiple force plots together to create a stacked force plot. Here we pass all SHAP values in the force plot function; though we can limit it. Each individual force plot is now vertical and stacked side by side. Copy/paste the content from the cell below to your notebook. The questions refer to this code cell and to the resulting plot.

1 - Run the cell and point out the value for explainer.expected_value[0].

2 - Set the dropdown at the top of the figure and to the left of the figure to 'Overall Qual'. What do you conclude from the resulting curves?

In [ ]:
# Plot stacked force plot.
shap.plots.force(
    
        base_value    = explainer.expected_value[0], # Mean prediction for the entire training data.
        shap_values   = shap_values,                 # SHAP values.
        features      = df_X_fraction,               # Training data.
        feature_names = df_X_fraction.columns        # Feature names.
)

In [ ]:
# ANSWERS

# 1 - The mean prediction (in my case 12.01) can be seen where the blue polygon hits the y-axis.

# 2 - We observe that as 'Overall Qual' increases, the SHAP value for Overall Qual increases.
# In other words, houses with higher overall quality tend to have higher sales prices.

### Exercise H - Mean SHAP

This next plot will tell us which features are most important. For each feature, we calculate the mean SHAP value across all observations. Specifically, we take the mean of the absolute values as we do not want positive and negative values to offset each other. There is one bar for each feature. Copy/paste the content from the cell below to your notebook. The questions refer to this code cell and to the resulting plot.

1 - Run the cell. What do you conclude from the resulting chart?

In [ ]:
# Plot Mean SHAP.
shap.plots.bar(

    shap_values = shap.Explanation(
        base_values   = explainer.expected_value[0], # Mean prediction for the entire training data.
        values        = shap_values,                 # Subset of shap values.
        data          = df_X_fraction,               # Subset of training data.
        feature_names = df_X_fraction.columns        # Feature names.
))



In [ ]:
# ANSWERS

# 1 - Features that have made large positive/negative contributions will have a large mean SHAP value.
# In other words, these are the features that have had a significant impact on the model’s predictions.
# In this sense, this plot can be used in the same way as a feature importance plot.
# Overall Qual is the biggest explainer of the sale price, by far.

### Exercise I - Beeswarm Plot

Next, we have the single most useful plot. The beeswarm visualises all of the SHAP values. Copy/paste the content from the cell below to your notebook. The questions refer to this code cell and to the resulting plot.

1 - Run the cell. What do you conclude from the resulting chart?

In [ ]:
# Plot beeswarm plot.
shap.plots.beeswarm(
    
    shap.Explanation(
        base_values   = explainer.expected_value[0], # Mean prediction for the entire training data.
        values        = shap_values,                 # SHAP values.
        data          = df_X_fraction,               # Training data.
        feature_names = df_X_fraction.columns        # Feature names.
))

In [ ]:
# ANSWERS

# 1 - On the y-axis, the values are grouped by feature. For each group, the colour of the points is
# determined by the feature value (i.e. higher feature values are redder).
# We can also start to understand the nature of these relationships. For Overall Quality, notice how as
# the feature value increases the SHAP values increase. We saw a similar relationship in the stacked
# force plot. It tells us that larger values for Overall Quality will lead to a higher predicted Sale Price.
